In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',
                                                         train = True,
                                                         download = True,
                                                         transform = transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307, ), (0.3081,))
                                                         ])),
                                          batch_size = BATCH_SIZE, shuffle = True)

In [5]:
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./.data',
                                                        train = False,
                                                        transform = transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize((0.1307, ), (0.3031,))
                                                        ])),
                                         batch_size = BATCH_SIZE, shuffle = True)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

In [7]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [8]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))


In [11]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.586661
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.228047
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.343396
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.393107
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.360426
[1] Test Loss: 0.1244, Accuracy: 96.13%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.306252
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.424817
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.239059
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.208025
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.172546
[2] Test Loss: 0.0957, Accuracy: 97.08%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.312127
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.372096
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.363630
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.138410
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.268174
[3] Test Loss: 0.0820, Accuracy: 97.40%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.288814
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.589109
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.219101
